In [1]:
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

#聲音處理套件
import librosa
#資料處理套件
import pandas as pd
import numpy as np
#時間套件
import time

from enum import Enum

In [2]:
#存檔位置
ModelMode = "LeNet5_ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
filePath_Train = ModelMode + TrainDataSoundMode[3] + "_EpochUp_LRup_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[3] +  "_EpochUp_LRup_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[3] + "_EpochUp_LRup"

In [3]:
#音訊資訊CSV檔位置
DATA_CSV_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/meta/esc50.csv"
#音訊存放位置
DATA_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/audio/"

def loadDataESC50(data_path, data_csv_path, soundLength):
    dataX_Fold = []
    dataY_Fold = []
    
    #讀取CSV檔
    df = pd.read_csv(data_csv_path)

    #計數
    count = 0
    
    for Fold in range(1, 6, 1):
        #初始化
        dataX = []
        dataY = []
        #篩選資料
        sound_df = df[df.fold == Fold]
        #轉換train資料/添加標籤
        for data in sound_df.itertuples():
            sig, sr = librosa.load(data_path+data[1])
            for i in range((6-soundLength)):
                sound = sig[i*sr : int((i+soundLength)*sr)]
                sound = audio_to_melspectrogram(audio=sound, sr=sr)
                dataX.append(sound)
                dataY.append(data[3])       
            count +=1
            print("{:.3f}".format(100 * count / len(df))+"%", end='\r')
        dataX_Fold.append(dataX)
        dataY_Fold.append(dataY)
                
    #通通轉成 torch 的 tensor 形式
    dataX_Fold = torch.Tensor(dataX_Fold)
    dataY_Fold = torch.Tensor(dataY_Fold).type(torch.LongTensor)
    
    #轉維度 FBCHW
    dataX_Fold = torch.reshape(dataX_Fold, (dataX_Fold.shape[0], dataX_Fold.shape[1], 1, dataX_Fold.shape[2], dataX_Fold.shape[3]))
           
    # 完成時打印最終消息
    print("Progress: 100% - Completed")
    return dataX_Fold, dataY_Fold

class conf:
    sr = 44100  #採樣率
    hop_length = 512  #帧移
    fmin = 20  #最低頻率
    fmax = sr // 2  #最高頻率
    n_mels = 128  #濾波器個數
    n_fft = 2048  #FFT窗口長度

#轉頻譜圖
def audio_to_melspectrogram(audio, sr):
    spectrogram = librosa.feature.melspectrogram(y=audio, 
                                                 sr=sr,
                                                 n_mels=128,
                                                 hop_length=512,
                                                 n_fft=2048,
                                                 fmin=20,
                                                 fmax=sr//2)
    spectrogram = librosa.power_to_db(spectrogram)
    return spectrogram

class CustomDatasetESC50(Dataset):
    def __init__(self, inputDataX, inputDataY, fold, isTrain):
        #防止範圍錯誤
        if(fold>5 or fold <1):
            raise ValueError("Fold範圍錯誤")
            
        #資料初始化
        self.dataX = []
        self.dataY = []
        
        # True:訓練資料  False:測試資料
        if isTrain:
            for Fold in range(1, 6, 1):
                if(Fold != fold):
                    for X in inputDataX[Fold-1]:
                        self.dataX.append(X)
                    for Y in inputDataY[Fold-1]:
                        self.dataY.append(Y)
        else:
            self.dataX = inputDataX[fold-1]
            self.dataY = inputDataY[fold-1]

    def __len__(self):
        return len(self.dataX)
    
    def __getitem__(self, index):
        # 根據索引返回單個樣本及其標籤
        data = self.dataX[index]
        label = self.dataY[index]
        
        return data, label

In [4]:
#LeNet5模型
class LeNet(nn.Module):
    def __init__(self, nums_class=10):
        super(LeNet,self).__init__()
        
        #卷基層
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        
        #全連接層
        self.fullconn_layer = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(32,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,nums_class)           
        )
        
    def forward(self,x):
        output = self.conv_layer(x)
        output = self.fullconn_layer(output)
        return output

In [5]:
#訓練模型(全部訓練)
def train_model_TrainAll(trainFold, setLR, setEpoch):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    
    #---------------------------------------------------
    #訓練相關參數
    #---------------------------------------------------
    #訓練次數
    num_epochs = setEpoch
    #批次大小
    BATCH_SIZE = 64
    #學習率
    LR = setLR
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    #優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #scheduler
    #--------------------------------------------------------------------
    #調度器
    scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #資料抓入
    #預處理
    #--------------------------------------------------------------------
    #資料抓入
    trainloader = CustomDatasetESC50(inputDataX = dataX,
                               inputDataY = dataY,
                               fold = trainFold,
                               isTrain = True)
    #分批方法
    train_loader = torch.utils.data.DataLoader(trainloader, batch_size = BATCH_SIZE, shuffle = True)
    #--------------------------------------------------------------------
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(inputs)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy))
        #調度器走一步
        scheduler.step()
        
    #測試準確度
    test_accuracy1 = test_model(inputDataX = dataX1, inputDataY = dataY1, testFold = trainFold)
    test_accuracy3 = test_model(inputDataX = dataX3, inputDataY = dataY3, testFold = trainFold)
    test_accuracy5 = test_model(inputDataX = dataX5, inputDataY = dataY5, testFold = trainFold)
        
    #將gpu資料取回
    train_loss_array = torch.stack(train_loss_array,dim=-1).cpu()
    train_accuracy_array = torch.stack(train_accuracy_array,dim=-1).cpu()
    
    return train_loss_array, train_accuracy_array, test_accuracy1, test_accuracy3, test_accuracy5

In [6]:
#測試模型的準確度
def test_model(inputDataX, inputDataY, testFold):
    #固定模型不改變權重
    model.eval()
    #計數初始化
    correct = 0
    total = 0
    #資料抓取
    testloader = CustomDatasetESC50(inputDataX = inputDataX,
                               inputDataY = inputDataY,
                               fold = testFold,
                               isTrain = False)
    #封裝                                
    test_loader = torch.utils.data.DataLoader(testloader, batch_size=64, shuffle=False)
    
    #不計算梯度 加速計算
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #預測結果
            outputs = model(inputs)
            
            #獲取預測標籤
            _, predicted = torch.max(outputs.data, 1)
            
            #計算準確度
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
    return (100 * correct / total)

In [7]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
dataX1, dataY1 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 1)
print("讀取3秒資料")
dataX3, dataY3 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 3)
print("讀取5秒資料")
dataX5, dataY5 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_16428\2625783892.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
Data Loading Time: 134.22 seconds


In [8]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = LeNet(nums_class = 50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    #學習率
    lr=0
    #epoch
    epoch=0
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        #訓練資料(要用幾秒訓練)
        if(TL==0):
            #1s
            dataX = dataX1
            dataY = dataY1
            epoch = 20
            lr=0.0008
        elif(TL==1):
            #3s
            dataX = dataX3
            dataY = dataY3
            epoch = 40
            lr=0.0016
        else:
            #5s
            dataX = dataX5
            dataY = dataY5
            epoch = 60
            lr=0.0032

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型(只訓練分類器)
        train_loss, train_accuracy, test_accuracy1, test_accuracy3, test_accuracy5 = train_model_TrainAll(trainFold=fold, setLR=lr, setEpoch=epoch)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/20 Train_Loss: 3.4265496730804443 Train_acc: 5.487500%
Train Epoch: 2/20 Train_Loss: 3.2220349311828613 Train_acc: 10.275001%
Train Epoch: 3/20 Train_Loss: 2.957651376724243 Train_acc: 15.825001%
Train Epoch: 4/20 Train_Loss: 2.9254369735717773 Train_acc: 18.862501%
Train Epoch: 5/20 Train_Loss: 2.7929494380950928 Train_acc: 22.325001%
Train Epoch: 6/20 Train_Loss: 2.505251169204712 Train_acc: 24.637501%
Train Epoch: 7/20 Train_Loss: 2.526228904724121 Train_acc: 27.687502%
Train Epoch: 8/20 Train_Loss: 2.2747058868408203 Train_acc: 29.975002%
Train Epoch: 9/20 Train_Loss: 2.183166265487671 Train_acc: 31.437502%
Train Epoch: 10/20 Train_Loss: 2.2052385807037354 Train_acc: 32.950001%
Train Epoch: 11/20 Train_Loss: 2.11630

Train Epoch: 41/60 Train_Loss: 0.38502195477485657 Train_acc: 91.875000%
Train Epoch: 42/60 Train_Loss: 0.2141239494085312 Train_acc: 94.125000%
Train Epoch: 43/60 Train_Loss: 0.2994219958782196 Train_acc: 92.625000%
Train Epoch: 44/60 Train_Loss: 0.22984668612480164 Train_acc: 92.750000%
Train Epoch: 45/60 Train_Loss: 0.1458454430103302 Train_acc: 92.250000%
Train Epoch: 46/60 Train_Loss: 0.1718713343143463 Train_acc: 95.062500%
Train Epoch: 47/60 Train_Loss: 0.11440781503915787 Train_acc: 96.812500%
Train Epoch: 48/60 Train_Loss: 0.1547335982322693 Train_acc: 96.250000%
Train Epoch: 49/60 Train_Loss: 0.17675139009952545 Train_acc: 96.375000%
Train Epoch: 50/60 Train_Loss: 0.1377170979976654 Train_acc: 96.750000%
Train Epoch: 51/60 Train_Loss: 0.2368105947971344 Train_acc: 96.562500%
Train Epoch: 52/60 Train_Loss: 0.1352834552526474 Train_acc: 97.062500%
Train Epoch: 53/60 Train_Loss: 0.10144179314374924 Train_acc: 96.500000%
Train Epoch: 54/60 Train_Loss: 0.1996074765920639 Train_acc

Train Epoch: 20/60 Train_Loss: 0.3892248272895813 Train_acc: 82.687500%
Train Epoch: 21/60 Train_Loss: 0.3847212493419647 Train_acc: 85.000000%
Train Epoch: 22/60 Train_Loss: 0.5428363084793091 Train_acc: 86.562500%
Train Epoch: 23/60 Train_Loss: 0.5252291560173035 Train_acc: 86.250000%
Train Epoch: 24/60 Train_Loss: 0.6259698867797852 Train_acc: 86.125000%
Train Epoch: 25/60 Train_Loss: 0.3617883324623108 Train_acc: 87.125000%
Train Epoch: 26/60 Train_Loss: 0.37999483942985535 Train_acc: 87.062500%
Train Epoch: 27/60 Train_Loss: 0.6588814854621887 Train_acc: 83.875000%
Train Epoch: 28/60 Train_Loss: 0.45429593324661255 Train_acc: 82.875000%
Train Epoch: 29/60 Train_Loss: 0.4501098096370697 Train_acc: 85.562500%
Train Epoch: 30/60 Train_Loss: 0.457228422164917 Train_acc: 87.625000%
Train Epoch: 31/60 Train_Loss: 0.2823171615600586 Train_acc: 87.187500%
Train Epoch: 32/60 Train_Loss: 0.2737375497817993 Train_acc: 92.250000%
Train Epoch: 33/60 Train_Loss: 0.20825797319412231 Train_acc: 9

Train Epoch: 1/60 Train_Loss: 2.310673475265503 Train_acc: 36.187500%
Train Epoch: 2/60 Train_Loss: 1.6309038400650024 Train_acc: 47.750000%
Train Epoch: 3/60 Train_Loss: 1.0278748273849487 Train_acc: 60.562500%
Train Epoch: 4/60 Train_Loss: 1.7571156024932861 Train_acc: 63.250000%
Train Epoch: 5/60 Train_Loss: 1.0335642099380493 Train_acc: 63.062500%
Train Epoch: 6/60 Train_Loss: 1.131085753440857 Train_acc: 62.062500%
Train Epoch: 7/60 Train_Loss: 1.3939777612686157 Train_acc: 56.500000%
Train Epoch: 8/60 Train_Loss: 1.0497417449951172 Train_acc: 61.437500%
Train Epoch: 9/60 Train_Loss: 0.8175497651100159 Train_acc: 66.187500%
Train Epoch: 10/60 Train_Loss: 0.8662785291671753 Train_acc: 73.312500%
Train Epoch: 11/60 Train_Loss: 0.6647124886512756 Train_acc: 72.000000%
Train Epoch: 12/60 Train_Loss: 0.7541033625602722 Train_acc: 69.125000%
Train Epoch: 13/60 Train_Loss: 1.4931286573410034 Train_acc: 69.437500%
Train Epoch: 14/60 Train_Loss: 1.186885952949524 Train_acc: 62.500000%
Trai

Train Epoch: 25/40 Train_Loss: 1.1646568775177002 Train_acc: 72.000000%
Train Epoch: 26/40 Train_Loss: 0.9887266159057617 Train_acc: 70.937500%
Train Epoch: 27/40 Train_Loss: 0.8927689790725708 Train_acc: 73.541672%
Train Epoch: 28/40 Train_Loss: 1.3259334564208984 Train_acc: 72.104172%
Train Epoch: 29/40 Train_Loss: 0.6963489651679993 Train_acc: 73.854172%
Train Epoch: 30/40 Train_Loss: 0.8408229947090149 Train_acc: 74.541672%
Train Epoch: 31/40 Train_Loss: 0.6465548872947693 Train_acc: 77.687500%
Train Epoch: 32/40 Train_Loss: 0.47421589493751526 Train_acc: 78.604172%
Train Epoch: 33/40 Train_Loss: 0.9628596305847168 Train_acc: 79.270836%
Train Epoch: 34/40 Train_Loss: 0.8445667624473572 Train_acc: 78.875000%
Train Epoch: 35/40 Train_Loss: 0.7458047866821289 Train_acc: 79.395836%
Train Epoch: 36/40 Train_Loss: 0.6092750430107117 Train_acc: 79.604172%
Train Epoch: 37/40 Train_Loss: 0.9882647395133972 Train_acc: 79.937500%
Train Epoch: 38/40 Train_Loss: 0.7253332138061523 Train_acc: 78

Train Epoch: 3/40 Train_Loss: 1.7176223993301392 Train_acc: 51.416668%
Train Epoch: 4/40 Train_Loss: 1.7999407052993774 Train_acc: 51.520836%
Train Epoch: 5/40 Train_Loss: 1.5431818962097168 Train_acc: 52.041668%
Train Epoch: 6/40 Train_Loss: 1.2220834493637085 Train_acc: 54.750000%
Train Epoch: 7/40 Train_Loss: 1.3659422397613525 Train_acc: 56.875000%
Train Epoch: 8/40 Train_Loss: 1.6553219556808472 Train_acc: 56.458336%
Train Epoch: 9/40 Train_Loss: 1.298024296760559 Train_acc: 55.520836%
Train Epoch: 10/40 Train_Loss: 1.3517347574234009 Train_acc: 57.583336%
Train Epoch: 11/40 Train_Loss: 1.3217558860778809 Train_acc: 57.875000%
Train Epoch: 12/40 Train_Loss: 1.112385869026184 Train_acc: 61.229168%
Train Epoch: 13/40 Train_Loss: 1.131150484085083 Train_acc: 62.291668%
Train Epoch: 14/40 Train_Loss: 1.246273398399353 Train_acc: 62.958336%
Train Epoch: 15/40 Train_Loss: 1.0989474058151245 Train_acc: 62.229168%
Train Epoch: 16/40 Train_Loss: 1.1419992446899414 Train_acc: 68.895836%
Tra

In [9]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()